Import necessary libraries

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from cancercnn import CancerCNN

In [35]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

Hyperparameters

In [36]:
image_size = 224
rotation = 10
normalization_mean = [0.5, 0.5, 0.5]
normalization_std = [0.5, 0.5, 0.5]

learning_rate = 0.03
batch_size = 32
epochs = 40

Image processing steps

In [37]:
train_transform = transforms.Compose([
    # Resize the images 
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),  # Flip the images horizontally at random
    # Rotate the images
    transforms.RandomRotation(rotation),
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    # Normalize the pixel values to [-1, 1]
    transforms.Normalize(normalization_mean, normalization_std)
])
test_transform = transforms.Compose([
    # Resize the images to 224x224 pixels
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),  # Convert the images to PyTorch tensors
    # Normalize the pixel values to [-1, 1]
    transforms.Normalize(normalization_mean, normalization_std)
])

In [38]:
# Load the dataset
train_dataset = datasets.ImageFolder('data/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)

Define the loss function and optimizer

In [39]:
model = CancerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [40]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)

Train The model

In [41]:
for epoch in range(epochs):
    train_loss = 0
    train_correct = 0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset), train_acc))

    # Test the model
    model.eval()
    test_loss = 0
    test_correct = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset), test_acc))

    # Calculate F1 score
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    f1_score = metrics.f1_score(all_targets, all_preds, average='macro')
    print('F1 score: {:.4f}'.format(f1_score))

    print('------------------------------------------------------')


Train set: Average loss: 0.0197, Accuracy: 1720/2637 (65%)
Test set: Average loss: 0.0251, Accuracy: 397/660 (60%)

F1 score: 0.4896
------------------------------------------------------
Train set: Average loss: 0.0160, Accuracy: 1988/2637 (75%)
Test set: Average loss: 0.0141, Accuracy: 511/660 (77%)

F1 score: 0.7730
------------------------------------------------------
Train set: Average loss: 0.0143, Accuracy: 2031/2637 (77%)
Test set: Average loss: 0.0135, Accuracy: 509/660 (77%)

F1 score: 0.7711
------------------------------------------------------
Train set: Average loss: 0.0134, Accuracy: 2082/2637 (79%)
Test set: Average loss: 0.0118, Accuracy: 543/660 (82%)

F1 score: 0.8227
------------------------------------------------------
Train set: Average loss: 0.0127, Accuracy: 2095/2637 (79%)
Test set: Average loss: 0.0123, Accuracy: 540/660 (82%)

F1 score: 0.8176
------------------------------------------------------
Train set: Average loss: 0.0120, Accuracy: 2138/2637 (81%)
T

### Save the model

In [43]:
torch.save(model.state_dict(), 'cancer_cnn.pth')